In [ ]:
!pip install transformers==4.51.3 nltk evaluate tqdm bert_score wandb
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [ ]:
from tqdm import tqdm
import sys
import os
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import corpus_bleu
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from datasets import load_dataset
import random
import numpy as np
# from transformers_models.marian.marianMT import


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
SEED_VALUE = 42


random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)

In [ ]:
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

#GETS SYNONYM FOR A WORD

def get_synonym(word):
    synonyms = wordnet.synsets(word)
    if not synonyms:
        return None

    lemmas = synonyms[0].lemmas()
    for lemma in lemmas:
        synonym = lemma.name().replace("_", " ")
        if synonym.lower() != word.lower():
            return synonym
    return None


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#GETS THE NEIGHBORS OF A QWERTY KEYBOARD
def build_qwerty_neighbors():
    layout = [
        "qwertyuiop",
        "asdfghjkl",
        "zxcvbnm"
    ]
    neighbors = {}

    for row in layout:
        for i, char in enumerate(row):
            neighbor_chars = []
            if i > 0:
                neighbor_chars.append(row[i - 1])
            if i < len(row) - 1:
                neighbor_chars.append(row[i + 1])
            neighbors[char] = ''.join(neighbor_chars)

    return neighbors

QWERTY_NEIGHBORS = build_qwerty_neighbors()


In [ ]:
#INTRODUCE TYPOS
def typo_char(c):
    if c.lower() in QWERTY_NEIGHBORS:
        return random.choice(QWERTY_NEIGHBORS[c.lower()])
    return c

In [ ]:

def add_noise(text, noise_prob):
    words = text.split()
    noisy_words = []

    for word in words:
        if random.random() < noise_prob:
            noise_type = random.choice(["delete_word", "duplicate_word", "shuffle", "synonym", "char_noise"])

            if noise_type == "delete_word":
                continue
            elif noise_type == "punctuation_insert":
                noisy_word = word + " " + random.choice([".", ",", "!", "?", ";"])
                noisy_words.append(noisy_word)
            elif noise_type == "char_noise":
                noisy_word = ""
                for char in word:
                    if random.random() < noise_prob:
                        char_noise_type = random.choice(["replace", "shuffle", "delete_char", "add_char", "typo"])
                        if char_noise_type == "typo":
                            noisy_word += typo_char(char)
                        elif char_noise_type == "add_char":
                            noisy_word += random.choice("abcdefghijklmnopqrstuvwxyz")
                            noisy_word += char
                        elif char_noise_type == "delete_char":
                            continue
                        elif char_noise_type == "shuffle":
                            char_list = list(char)
                            random.shuffle(char_list)
                            noisy_word += ''.join(char_list)
                        elif char_noise_type == "replace":
                            noisy_word += random.choice("abcdefghijklmnopqrstuvwxyz")
                    else:
                        noisy_word += char
                noisy_words.append(noisy_word)
        else:
            noisy_words.append(word)

    return " ".join(noisy_words)

In [ ]:
def retrieve_data(max_length, add_text_noise, noise_level):
    dataset = load_dataset("wmt14", "de-en")

    raw_subset = dataset["train"].select(range(200000))

    def is_short(example):
        return len(example["translation"]["de"].split()) <= max_length and len(example["translation"]["en"].split()) <= max_length

    filtered = raw_subset.filter(is_short)

    if add_text_noise:
        def apply_noise(example):
            example["translation"]["en"] = add_noise(example["translation"]["en"], noise_level)
            return example

        filtered = filtered.map(apply_noise)

    train_data = filtered.select(range(50000))
    val_data = filtered.select(range(50000, 53000))
    test_data = filtered.select(range(53000, 56000))

    return {
        "train": train_data,
        "validation": val_data,
        "test": test_data
    }

In [ ]:
def translated(n, model):
    return model.translate_text(n)

In [ ]:
data = retrieve_data(max_length= 50, add_text_noise=True, noise_level=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/185225 [00:00<?, ? examples/s]

In [ ]:
print(data["train"].shape)
print(data["test"].shape)
print(data["validation"].shape)

(50000, 1)
(3000, 1)
(3000, 1)


In [ ]:
data["validation"]["translation"][:10]

[{'de': 'Der Europäische Forschungsraum ...', 'en': 'So the area ...'},
 {'de': '(Der Präsident entzieht dem Redner das Wort.)',
  'en': '(The cut speaker off)'},
 {'de': 'Herr Präsident, ich möchte Frau Plooij-van Gorsel danken, denn sie hat wieder einmal einen ausgezeichneten Bericht vorgelegt.',
  'en': 'Mr President, I wish thank Mrs Plooij-Van Gorsel her very good report.'},
 {'de': 'Ich danke Herrn Busquin, der diese Debatte in Gang gesetzt und dafür den richtigen Zeitpunkt gewählt hat.',
  'en': 'I thank the whose timing very good idea of launching this debate.'},
 {'de': 'Tatsächlich wächst das Unbehagen über die mangelnde Koordinierung im europäischen Wissenschafts- und Forschungsbereich.',
  'en': 'There is indeed a growing feeling of unease at the lack of coordination in European science and research.'},
 {'de': 'Die Reaktionen sind positiv, wie Herr Busquin aufgrund seiner Kontakte mit den entsprechenden Stellen in Europa weiß und wie er auch bei einem kürzlichen Besuch in 

In [ ]:
data["test"]["translation"][:10]

[{'de': 'In der französischen Version kommt der Begriff "délit " in der englischen dagegen der Begriff "crime " vor.',
  'en': 'The French versuin contains the word whereas this is written as "crime" in the version.'},
 {'de': 'Wie Sie wissen, hat der französische Begriff nicht dieselbe Bedeutung.',
  'en': 'As you know, President, tiis does have the meaning in French.'},
 {'de': 'Mit dem Änderungsantrag soll diese Version berichtigt werden.',
  'en': 'The amendment to correct this.'},
 {'de': 'Ich habe ihn zwar nicht eingereicht, aber ich denke, dies ist sein Zweck.',
  'en': 'I am not the one who tabled the amendment, but it to me that this is the point of'},
 {'de': 'Darauf wollte ich hinweisen.', 'en': 'I point this out.'},
 {'de': 'Zum Änderungsantrag 5:', 'en': 'Relating to Amendment No 5'},
 {'de': 'Frau Präsidentin! In den Gesprächen mit Frau Martens von der PPE-Fraktion haben wir beschlossen, den Änderungsantrag zu Ziffer 19 in diesem Sinne zu ändern.',
  'en': 'President, in 

In [ ]:
data["train"]["translation"][:10]

[{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption the'},
 {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'en': 'I declare session of European Parliament adjourned on Friday 17 December and once again to wish you a happy new year in the hope you enjoyed a pleasant festive period.'},
 {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
  'en': "Although, you will have seen, the bug' failed to materialise, still the in a number of countries suffered a series of natural disasters that truly dreadful."},
 {'de': 'Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.',
  'en': 'have requested debate 

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch

model_name1 = "Helsinki-NLP/opus-mt-en-de"
marian_tokenizer = MarianTokenizer.from_pretrained(model_name1)
marian_model = MarianMTModel.from_pretrained(model_name1)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from transformers import M2M100Model, M2M100Tokenizer, M2M100ForConditionalGeneration

model_name2 = "facebook/m2m100_418M"
m2m100_tokenizer = M2M100Tokenizer.from_pretrained(model_name2)
m2m100_model =  M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
english = [n["en"] for n in data["train"]["translation"]]
german = [n["de"] for n in data["train"]["translation"]]

In [ ]:
data

{'train': Dataset({
     features: ['translation'],
     num_rows: 50000
 }),
 'validation': Dataset({
     features: ['translation'],
     num_rows: 3000
 }),
 'test': Dataset({
     features: ['translation'],
     num_rows: 3000
 })}

In [ ]:
def translate_with_model(model, tokenizer, src_texts, batch_size=16, device="cuda"):
    """
    Translates a list of source texts using the specified model and tokenizer.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    predictions = []

    for i in tqdm(range(0, len(src_texts), batch_size), desc="Translating"):
        batch = src_texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=60)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs, num_beams=4, max_length=60, early_stopping=True)
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(preds)

    return predictions

In [ ]:
m2m100_tokenizer.src_lang = "en"
m2m100_tokenizer.tgt_lang = "de"

def preprocess_marian(batch):
    src_texts = [ex["en"] for ex in batch["translation"]]
    tgt_texts = [ex["de"] for ex in batch["translation"]]

    model_inputs_marian = marian_tokenizer(
        src_texts,
        truncation=True,
        padding="max_length",
        max_length=40
    )

    with marian_tokenizer.as_target_tokenizer():
        labels = marian_tokenizer(
            tgt_texts,
            truncation=True,
            padding="max_length",
            max_length=40
        )["input_ids"]

    labels = [
        [(token if token != marian_tokenizer.pad_token_id else -100) for token in seq]
        for seq in labels
    ]

    model_inputs_marian["labels"] = labels
    return model_inputs_marian


In [ ]:
def preprocess_m2m100(batch):
    src_texts = [ex["en"] for ex in batch["translation"]]
    tgt_texts = [ex["de"] for ex in batch["translation"]]

    model_inputs_m2m100 = m2m100_tokenizer(
        src_texts,
        truncation=True,
        padding="max_length",
        max_length=40
    )

    with m2m100_tokenizer.as_target_tokenizer():
        labels = m2m100_tokenizer(
            tgt_texts,
            truncation=True,
            padding="max_length",
            max_length=40
        )["input_ids"]

    labels = [
        [(token if token != m2m100_tokenizer.pad_token_id else -100) for token in seq]
        for seq in labels
    ]

    model_inputs_m2m100["labels"] = labels
    return model_inputs_m2m100


In [ ]:
tokenized_data_marian = {
    "train": data["train"].map(preprocess_marian, batched = True),
    "validation": data["validation"].map(preprocess_marian, batched=True)
}


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_data_m2m100 = {
    "train": data["train"].map(preprocess_m2m100, batched = True),
    "validation": data["validation"].map(preprocess_m2m100, batched=True)
}


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
import evaluate
test_subset = data["test"].select(range(3000))
src_texts = [ex["translation"]["en"] for ex in data["test"]]
references = [[ex["translation"]["de"]] for ex in test_subset]

marian_predictions = translate_with_model(marian_model, marian_tokenizer, src_texts)

m2m_predictions = translate_with_model(m2m100_model, m2m100_tokenizer, src_texts)

bleu = evaluate.load("bleu")

models = [marian_model, m2m100_model]
tokenizers = [marian_tokenizer, m2m100_tokenizer]

for i, (model, tokenizer) in enumerate(zip(models, tokenizers), start=1):
    predictions = translate_with_model(model, tokenizer, src_texts)
    references = [[ref] for ref in references]

    bleu_score = bleu.compute(predictions=predictions, references=references)
    print(f"Model {i} BLEU Score: {bleu_score['bleu']:.4f}")

Translating: 100%|██████████| 188/188 [06:51<00:00,  2.19s/it]


Translating: 100%|██████████| 188/188 [01:40<00:00,  1.88it/s]


Model 1 BLEU Score: 0.1222


Translating: 100%|██████████| 188/188 [06:50<00:00,  2.18s/it]


Model 2 BLEU Score: 0.0087


In [ ]:
references[:10]

[[[['In der französischen Version kommt der Begriff "délit " in der englischen dagegen der Begriff "crime " vor.']]],
 [[['Wie Sie wissen, hat der französische Begriff nicht dieselbe Bedeutung.']]],
 [[['Mit dem Änderungsantrag soll diese Version berichtigt werden.']]],
 [[['Ich habe ihn zwar nicht eingereicht, aber ich denke, dies ist sein Zweck.']]],
 [[['Darauf wollte ich hinweisen.']]],
 [[['Zum Änderungsantrag 5:']]],
 [[['Frau Präsidentin! In den Gesprächen mit Frau Martens von der PPE-Fraktion haben wir beschlossen, den Änderungsantrag zu Ziffer 19 in diesem Sinne zu ändern.']]],
 [[['In diesem Änderungsantrag wird auf die wichtige Rolle der NRO hingewiesen, zu denen auch soziale Organisationen in Europa gehören können, die sich die Aufnahme und Betreuung der Opfer zur Aufgabe gemacht haben.']]],
 [[['Ich stehe zu diesem Änderungsantrag.']]],
 [[['Sie bringen also einen mündlichen Änderungsantrag ein, um die Worte "Kirchen und religiöse Organisationen " durch "soziale Organisati

In [ ]:
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
bleu_score_marian= bleu.compute(predictions=marian_predictions, references=references)
meteor_score_marian = meteor.compute(predictions=marian_predictions, references=[r[0] for r in references])
bert_score_marian = bertscore.compute(predictions=marian_predictions, references=[r[0] for r in references], lang="de")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
bleu_score_m2m100= bleu.compute(predictions=m2m_predictions, references=references)
meteor_score_m2m100 = meteor.compute(predictions=m2m_predictions, references=[r[0] for r in references])
bert_score_m2m100 = bertscore.compute(predictions=m2m_predictions, references=[r[0] for r in references], lang="de")

In [ ]:
def get_scores(bleu_score, meteor_score, bert_score):
    """Extracts and returns the scores as a dictionary."""
    bert_precision = sum(bert_score['precision']) / len(bert_score['precision'])
    bert_recall = sum(bert_score['recall']) / len(bert_score['recall'])
    bert_f1 = sum(bert_score['f1']) / len(bert_score['f1'])
    return {
        'BLEU': bleu_score['bleu'],
        'METEOR': meteor_score['meteor'],
        'Precision(BERT)': bert_precision,
        'Recall(BERT)': bert_recall,
        'F1(BERT)': bert_f1
    }

marian_scores = get_scores(bleu_score_marian, meteor_score_marian, bert_score_marian)
m2m100_scores = get_scores(bleu_score_m2m100, meteor_score_m2m100, bert_score_m2m100)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = pd.DataFrame({
    'Metric': marian_scores.keys(),
    'MarianMT': marian_scores.values(),
    'M2M100': m2m100_scores.values()
})

sns.set(style="white", context="talk")
palette = sns.color_palette("viridis", len(data))

plt.figure(figsize=(12, 6))
data_melted = data.melt(id_vars='Metric', var_name='Model', value_name='Score')
ax = sns.barplot(x='Metric', y='Score', hue='Model', data=data_melted, palette='viridis')

for i, bar in enumerate(ax.patches):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.02,
        f"{bar.get_height():.4f}",
        ha='center', va='bottom', fontsize=10
    )

plt.title("Evaluation Metrics Comparison with 20% noise", fontsize=18, pad=20)
plt.ylim(0, 1.1)
plt.ylabel("Score", fontsize=14)
plt.xlabel("")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.despine()
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)
plt.tight_layout()
plt.legend(title="Model", fontsize=12)
plt.show()